In [2]:
# importing libraries
import pandas as pd
import os
from dotenv import load_dotenv

# loading environment variables
load_dotenv()
DATA_PATH = os.getenv('DATA_PATH')

In [5]:
# loading data
df = pd.read_parquet(os.path.join(DATA_PATH, '../processed/dsi-qa-pairs.parquet'))
df.head()

,question,answer
0,"During the COVID-19 pandemic, ns and judgment ...",stress loss data and experienced the power of ...
1,mmons spent the past five years in the Risk Ma...,Sydney Simmons
2,What is the name of the inaugural recipient of...,Sydney Simmons
3,Women in Technology of Tennessee (WiTT) is exc...,Sydney Simmons the inaugural recipient of the ...
4,What is the name of the program at Vanderbilt?,Data Science Master’s ProgramThe data science ...
